In [12]:
import psycopg2

In [34]:
import pickle
#mypw="xxxxx"
#pickle.dump( mypw, open( "psqlPW.p", "wb" ) )
psqlPW = pickle.load( open( "psqlPW.p", "rb" ) )

In [35]:
connection = psycopg2.connect(database="pythonspatial", user="postgres", password=psqlPW, port="5433")

In [37]:
# connect to database and create a table
cursor = connection.cursor()
#cursor.execute("CREATE TABLE art_pieces (id serial PRIMARY KEY, code varchar(255), location GEOMETRY)")
#connection.commit

In [23]:
import requests, json

below we get a URL for a data feature. The 11 at the end is the ID of the data we want, in this case it is ground water monitoring sites. We will then create our query and define the the output type of json and the output fields. Finally we make our request. a resource for rest servers is here: https://mappingsupport.com/p/surf_gis/list-federal-state-county-city-GIS-servers.pdf


inFeatures = "https://geodata.state.nj.us/arcgis/rest/services/Features/Environmental/MapServer/11/query"
json_payload = { 'where': 'acres > 5','f': 'json', 'outFields': 'latitude,longitude,incidentname,acres'}
r = requests.get(inFeatures, params=json_payload)
print r.text

In [16]:
url='http://coagisweb.cabq.gov/arcgis/rest/services/public/PublicArt/MapServer/0/query'
params={"where":"1=1","outFields":"*","outSR":"4326","f":"json"}
r=requests.get(url, params=params)
data=r.json()


In [24]:
data["features"][0]

{'attributes': {'OBJECTID': 1095217,
  'ART_CODE': '101',
  'TITLE': 'Almond Blossom/Astronomy',
  'TYPE': 'public sculpture',
  'YEAR': '1986',
  'ARTIST': 'David Anderson',
  'ADDRESS': '4440 Osuna NE',
  'LOCATION': 'Osuna Median bet.Jefferson/ W.Frontage Rd',
  'X': -106.5918383,
  'Y': 35.1555,
  'IMAGE_URL': 'http://www.flickr.com/photos/abqpublicart/6831137393/',
  'JPG_URL': 'http://farm8.staticflickr.com/7153/6831137393_fa38634fd7_m.jpg'},
 'geometry': {'x': -106.59183830022498, 'y': 35.155500000061544}}

In [30]:
for a in data["features"]:
    code=a["attributes"]["ART_CODE"]
    wkt="POINT("+str(a["geometry"]["x"])+" "+str(a["geometry"]["y"])+")"
    if a["geometry"]["x"]=='NaN':
        pass
    else:
        cursor.execute("INSERT INTO art_pieces (code, location) VALUES({}, ST_GeomFromText('{}'))".format(code, wkt))
        
connection.commit()

In [38]:
connection.close()